In [17]:
from pymonad.tools import curry 
import sys
import random

# Pig simulation

## Strategies

$Score : \cal{N} \times \cal{N}$

$Strategy : Score \times \left\{Score\right\} \rightarrow bool$ 

### stick at n

In [10]:
@curry(3)
def stick_at_n(n,player_score,opponent_scores) :
    _,turn_score = player_score
    return True if turn_score < n else False
    

In [11]:
stick_at_20 = stick_at_n(20)
stick_at_20((71,20),{(89,0)})

False

## Dice

In [20]:
def dice_class(sides,seed) : # remember - a class is just function that returns an object
    def dice() : # the object produced by the class - remember, an object is just a function with memory
        nonlocal seed # the objects memory
        random.seed(seed)
        seed = random.randint(0,sys.maxsize) # note - python 3 specific
        dice = dice_class(sides,seed) # notice the reference to the enclosing function
        return dice,random.randint(1,sides)
    return dice
        

In [23]:
D6 = dice_class(6,0)
for i in range(10) :
    D6,score = D6()
    print(score)

4
6
2
4
6
3
3
1
1
2


## Turn



$Player : Strategy \times Score$

$Turn : Strategy \times Score \times \left\{Score\right\} \rightarrow Score$

In [59]:
@curry(5)
def pig_turn(dice,jeopardy_score,strategy,player_score,opponent_scores) :
    current_score = accumulated,turn = player_score
    while strategy(current_score,opponent_scores) :
        dice,roll = dice()
        if roll == jeopardy_score :
            return player_score
        turn += roll
        current_score = accumulated,turn
    return accumulated + turn,0
        
        

In [60]:
D6 = dice_class(6,0)
stick_at_10 = stick_at_n(10)
jeopardy_score = 1
turn = pig_turn(D6,jeopardy_score)

In [61]:
player_score = (0,0)
opponent_scores = {(0,0)}

for i in range(20) :
    print(player_score)
    player_score = turn(stick_at_10,player_score,opponent_scores)

(0, 0)
(10, 0)
(22, 0)
(34, 0)
(44, 0)
(56, 0)
(56, 0)
(56, 0)
(56, 0)
(56, 0)
(71, 0)
(84, 0)
(94, 0)
(106, 0)
(121, 0)
(135, 0)
(146, 0)
(159, 0)
(170, 0)
(180, 0)


## Round

$Win : Score \rightarrow Bool$

$Round : \left\{Strategy\right\} \rightarrow Strategy$ 

In [54]:
@curry(2)
def pig_win(target,score) :
    accumulated,turn = score
    return False if accumulated + turn < target else True
    

In [90]:
@curry(3)
def pig_round(win,turn,strategies) :
    players = {s : (0,0) for s in strategies}
    while True :
        for strategy in players :
            score = turn(strategy,players[strategy],{players[s] for s in strategies if s != strategy})
            if win(score) : return strategy
            players[strategy] = score

In [148]:
win = pig_win(50)
D6 = dice_class(6,0)
jeopardy_score = 1
turn = pig_turn(D6,jeopardy_score)
stick_at_10 = stick_at_n(10)
stick_at_15 = stick_at_n(15)
strategies = (stick_at_10,stick_at_15)

In [149]:
around = pig_round(win,turn)

In [150]:
around(strategies) == stick_at_15

True

## Match

$Match : \left\{Strategy\right\} \rightarrow (Strategy \rightarrow \cal{N})$

In [151]:
@curry(3)
def pig_competition(around,n,strategies) :
    results = {s : 0 for s in strategies}
    for i in range(n) :
        s = around(strategies)
        results[around(strategies)] += 1
    return results

In [152]:
competition = pig_competition(around,100)
results = competition(strategies)

In [153]:
results[stick_at_15]

53

## Tournament

$Tournament : \left\{Strategy\right\} \rightarrow (Strategy^{n} \rightarrow \cal{R}^{n})$

In [168]:
@curry(3)
def pig_tournament(nplayers,competition,strategies) : 
    return {s : competition(s) for s in itertools.product(strategies,repeat=nplayers)}
    

In [169]:
tournament = pig_tournament(3,competition)

In [170]:
results = tournament(strategies)

In [165]:
results

{(<function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>,
  <function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>,
  <function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>): {<function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>: 100},
 (<function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>,
  <function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>,
  <function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>): {<function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>: 46,
  <function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>: 54},
 (<function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>,
  <function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any

In [125]:
import itertools

def f(x) :
    a,b = x
    return a+b

S = {1,2,3,4}
# list(itertools.product(S,repeat=3))

{s : None for s in itertools.product(S,repeat=3)}

{(1, 1, 1): None,
 (1, 1, 2): None,
 (1, 1, 3): None,
 (1, 1, 4): None,
 (1, 2, 1): None,
 (1, 2, 2): None,
 (1, 2, 3): None,
 (1, 2, 4): None,
 (1, 3, 1): None,
 (1, 3, 2): None,
 (1, 3, 3): None,
 (1, 3, 4): None,
 (1, 4, 1): None,
 (1, 4, 2): None,
 (1, 4, 3): None,
 (1, 4, 4): None,
 (2, 1, 1): None,
 (2, 1, 2): None,
 (2, 1, 3): None,
 (2, 1, 4): None,
 (2, 2, 1): None,
 (2, 2, 2): None,
 (2, 2, 3): None,
 (2, 2, 4): None,
 (2, 3, 1): None,
 (2, 3, 2): None,
 (2, 3, 3): None,
 (2, 3, 4): None,
 (2, 4, 1): None,
 (2, 4, 2): None,
 (2, 4, 3): None,
 (2, 4, 4): None,
 (3, 1, 1): None,
 (3, 1, 2): None,
 (3, 1, 3): None,
 (3, 1, 4): None,
 (3, 2, 1): None,
 (3, 2, 2): None,
 (3, 2, 3): None,
 (3, 2, 4): None,
 (3, 3, 1): None,
 (3, 3, 2): None,
 (3, 3, 3): None,
 (3, 3, 4): None,
 (3, 4, 1): None,
 (3, 4, 2): None,
 (3, 4, 3): None,
 (3, 4, 4): None,
 (4, 1, 1): None,
 (4, 1, 2): None,
 (4, 1, 3): None,
 (4, 1, 4): None,
 (4, 2, 1): None,
 (4, 2, 2): None,
 (4, 2, 3): None,
 (4, 2, 4)